In [24]:
import sqlite3

conn = sqlite3.connect('orgdb.sqlite')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS Counts')

cur.execute('''
CREATE TABLE Counts (org TEXT, count INTEGER)''')

fname = input('Enter file name: ')
if (len(fname) < 1): fname = 'mbox-short.txt'
fh = open(fname)
for line in fh:
    if not line.startswith('From: '): continue
    pieces = line.split()
    domain = pieces[1]
    email = domain.split('@')
    org = email[-1]
    cur.execute('SELECT count FROM Counts WHERE org = ? ', (org,))
    row = cur.fetchone()
    if row is None:
        cur.execute('''INSERT INTO Counts (org, count)
                VALUES (?, 1)''', (org,))
    else:
        cur.execute('UPDATE Counts SET count = count + 1 WHERE org = ?',
                    (org,))
    conn.commit()

# https://www.sqlite.org/lang_select.html
sqlstr = 'SELECT org, count FROM Counts ORDER BY count DESC LIMIT 10'

for row in cur.execute(sqlstr):
    print(str(row[0]), row[1])

cur.close()

Enter file name: mbox.txt
iupui.edu 536
umich.edu 491
indiana.edu 178
caret.cam.ac.uk 157
vt.edu 110
uct.ac.za 96
media.berkeley.edu 56
ufp.pt 28
gmail.com 25
et.gatech.edu 17


In [ ]:
import xml.etree.ElementTree as ET
import sqlite3

In [15]:
connection = sqlite3.connect('trackdb.sqlite')
cursor = connection.cursor()

# make some fresh tables 
cursor.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;

CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')

fname = input('enter: - ')
if len(fname) > 1 :
    fname = 'Library.xml'
    
def lookup(d, key):
    found = False
    for child in d:
        if found : return child.text
        if child.tag == 'key' and child.text == key :
            found = True
    return None
    
data = ET.parse(fname)
all_items = data.findall('dict/dict/dict')
print('Dict count:', len(all_items ))

for entry in all_items:
    if ( lookup(entry, 'Track ID') is None ) : continue

    name = lookup(entry, 'Name')
    artist = lookup(entry, 'Artist')
    album = lookup(entry, 'Album')
    count = lookup(entry, 'Play Count')
    rating = lookup(entry, 'Rating')
    length = lookup(entry, 'Total Time')
    
    if name is None or artist is None or album is None :
        continue 
        
    print(name, artist, album, count, rating , length)     
    
    cursor.execute('INSERT OR IGNORE INTO Artist (name) VALUES (?)', (artist,))
    
    cursor.execute('SELECT id FROM Artist WHERE name = ?', (artist,))
    artist_id = cursor.fetchone()[0]
    
    cursor.execute('INSERT OR IGNORE INTO Album (title, artist_id) VALUES (? ,?)',(album,artist_id))
    cursor.execute('SELECT id from Album WHERE title = ?',(album,))
    album_id = cursor.fetchone()[0]
    
    cursor.execute('''INSERT OR REPLACE INTO Track
        (title, album_id, len, rating, count) 
        VALUES ( ?, ?, ?, ?, ? )''', 
        ( name, album_id, length, rating, count ) )

    connection.commit()

enter: - Library.xml
Dict count: 404
Another One Bites The Dust Queen Greatest Hits 55 100 217103
Asche Zu Asche Rammstein Herzeleid 79 100 231810
Beauty School Dropout Various Grease 48 100 239960
Black Dog Led Zeppelin IV 109 100 296620
Bring The Boys Back Home Pink Floyd The Wall [Disc 2] 33 100 87118
Circles Bryan Lee Blues Is 54 60 355369
Comfortably Numb Pink Floyd The Wall [Disc 2] 36 100 384130
Crazy Little Thing Called Love Queen Greatest Hits 38 100 163631
Electric Funeral Black Sabbath Paranoid 44 100 293015
Fat Bottomed Girls Queen Greatest Hits 38 100 257515
For Those About To Rock (We Salute You) AC/DC Who Made Who 84 100 353750
Four Sticks Led Zeppelin IV 84 100 284421
Furious Angels Rob Dougan The Matrix Reloaded 54 100 330004
Gelle Bryan Lee Blues Is 45 60 199836
Going To California Led Zeppelin IV 100 100 215666
Grease Various Grease 42 100 205792
Hand of Doom Black Sabbath Paranoid 36 100 429609
Hells Bells AC/DC Who Made Who 82 100 312946
Hey You Pink Floyd The Wall

Try Your Best Kaiser Chiefs Yours Truly, Angry Mob 8 None 222511
Retirement Kaiser Chiefs Yours Truly, Angry Mob 10 None 237426
The Angry Mob [Live From Berlin] Kaiser Chiefs Yours Truly, Angry Mob 40 None 279066
I Like To Fight Kaiser Chiefs Yours Truly, Angry Mob 8 None 218566
From The Neck Down Kaiser Chiefs Yours Truly, Angry Mob 15 None 147226
Bomb Squad (TECH) Brent Brent's Album 4 None 208065
BYURY ME Brent Peanut Butter & Jam  4 None 274076
Charlie and the Rising Moon Charlie And The Rising Moon Charlie and The Rising Moon None None 161645
clay techno  Brent Brent's Album 5 None 276062
Cloud Nine Times Over  Charlie And The Rising Moon Charlie and The Rising Moon 3 None 135131
Depression in Session Brent Peanut Butter and Jam  4 None 213211
Heavy Brent Brent's Album 2 None 188055
Hi metal man Brent Brent's Album 4 None 260153
Mistro Brent Brent's Album 5 None 178076
Pirate spirit Brent Brent's Album 3 None 180480
Run Away (New)  Brent Brent's Album 1 None 154383
Star Gaze (Insp

The Rise of JavaScript IEEE Computer Society Computing Conversations None None 526132
Joseph Hardin: NCSA Mosaic IEEE Computer Society Computing Conversations None None 845635
Len Kleinrock on the Internet's First Two Packets IEEE Computer Society Computing Conversations 1 None 594390
Doug Van Houweling on Building the NSFNet IEEE Computer Society Computing Conversations None None 773146
Computing Conversations: Nathaniel Borenstein on MIME IEEE Computer Society Computing Conversations None None 682135
You Don't Mess Around With Jim Jim Croce Classic Hits None None 184398
Andrew S. Tanenbaum on MINIX IEEE Computer Society Computing Conversations None None 603000
Computing Conversations: Elizabeth Fong on SQL Standards IEEE Computer Society Computing Conversations None None 533577
Nii Quaynor on Bringing the Internet to Africa IEEE Computer Society Computing Conversations 1 None 673332
PHP-09-Transactions.mp3 Created by Sakai SI 664 W14's official Podcast. None None 1728888
PHP-01-Intro

In [2]:
import xml.etree.ElementTree as ET
import sqlite3

conn = sqlite3.connect('trackdb.sqlite')
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Genre; 
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;
CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);
CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);
CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);
CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')


fname = input('Enter file name: ')
if ( len(fname) < 1 ) : fname = 'Library.xml'

# <key>Track ID</key><integer>369</integer>
# <key>Name</key><string>Another One Bites The Dust</string>
# <key>Artist</key><string>Queen</string>

# loop thru all the childern in this dictionary and find a child tag that 
# has a particular key 

def lookup(d, key):
    found = False
    for child in d:
        if found : return child.text
        if child.tag == 'key' and child.text == key :
            found = True
    return None

stuff = ET.parse(fname)
all = stuff.findall('dict/dict/dict')

print( 'Dict count:', len(all))

for entry in all:
    if ( lookup(entry, 'Track ID') is None ) : continue

    name = lookup(entry, 'Name')
    artist = lookup(entry, 'Artist')
    album = lookup(entry, 'Album')
    #added this
    genre = lookup(entry,'Genre')
    count = lookup(entry, 'Play Count')
    rating = lookup(entry, 'Rating')
    length = lookup(entry, 'Total Time')

    if name is None or artist is None or album is None or genre is None: 
        continue

    print( name, artist, album, genre, count, rating, length)

    cur.execute('''INSERT OR IGNORE INTO Artist (name) 
        VALUES ( ? )''', ( artist, ) )
    cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
    artist_id = cur.fetchone()[0]
    #Added this
    cur.execute('''INSERT OR IGNORE INTO Genre (name) 
        VALUES ( ? )''', ( genre, ) )
    cur.execute('SELECT id FROM Genre WHERE name = ? ', (genre, ))
    genre_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
        VALUES ( ?, ? )''', ( album, artist_id ) )
    cur.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
    album_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Track
        (title, album_id, genre_id, len, rating, count) 
        VALUES ( ?, ?, ?, ?, ?, ? )''', 
        ( name, album_id, genre_id, length, rating, count ) )

    conn.commit()
#Added this
sqlstr = 'SELECT Track.title, Artist.name, Album.title, Genre.name FROM Track JOIN Genre JOIN Album JOIN Artist ON Track.genre_id = Genre.ID and Track.album_id = Album.id AND Album.artist_id = Artist.id ORDER BY Artist.name LIMIT 3'


for row in cur.execute(sqlstr) :
    print(str(row[0]),str(row[1]),str(row[2]),str(row[3]))

cur.close()

Enter file name: Library.xml
Dict count: 404
Another One Bites The Dust Queen Greatest Hits Rock 55 100 217103
Asche Zu Asche Rammstein Herzeleid Industrial 79 100 231810
Beauty School Dropout Various Grease Soundtrack 48 100 239960
Black Dog Led Zeppelin IV Rock 109 100 296620
Bring The Boys Back Home Pink Floyd The Wall [Disc 2] Rock 33 100 87118
Circles Bryan Lee Blues Is Funk 54 60 355369
Comfortably Numb Pink Floyd The Wall [Disc 2] Rock 36 100 384130
Crazy Little Thing Called Love Queen Greatest Hits Rock 38 100 163631
Electric Funeral Black Sabbath Paranoid Metal 44 100 293015
Fat Bottomed Girls Queen Greatest Hits Rock 38 100 257515
For Those About To Rock (We Salute You) AC/DC Who Made Who Rock 84 100 353750
Four Sticks Led Zeppelin IV Rock 84 100 284421
Furious Angels Rob Dougan The Matrix Reloaded Soundtrack 54 100 330004
Gelle Bryan Lee Blues Is Blues/R&B 45 60 199836
Going To California Led Zeppelin IV Rock 100 100 215666
Grease Various Grease Soundtrack 42 100 205792
Hand

Pahrump-Big Water Cusco Natural Wonders Music Sampler 1999 New Age 342 None 191634
Seeker's Quest Cheryl Gunn Natural Wonders Music Sampler 1999 New Age 337 None 236434
Floating To Forever Dean Everson Natural Wonders Music Sampler 1999 New Age 337 None 351686
Open Road Jeff Bailey Relaxing Jazz Jazz 10 None 318563
Ruby Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 13 None 205008
The Angry Mob Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 41 None 288313
Heat Dies Down Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 12 None 237061
Highroyds Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 10 None 199575
Love's Not A Competition (But I'm Winning) Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 11 None 197799
Thank You Very Much Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 11 None 157753
I Can Do It Without You Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 11 None 204199
My Kind Of Guy Kaiser Chiefs Yours Truly, Angry Mob Alt

Larry Smarr: Building Mosaic IEEE Computer Society Computing Conversations Podcast None None 894693
Len Kleinrock: The Theory of Packets IEEE Computer Society Computing Conversations Podcast 1 None 674742
Mitchell Baker: The Mozilla Foundation IEEE Computer Society Computing Conversations Podcast None None 994246
Pooja Sankar: Building the Piazza Collaboration System IEEE Computer Society Computing Conversations Podcast None None 496404
Van Jacobson: Content-Centric Networking IEEE Computer Society Computing Conversations Podcast None None 780251
The Apache Software Foundation IEEE Computer Society Computing Conversations Podcast None None 542484
A Brief History of Packets IEEE Computer Society Computing Conversations Podcast None None 1004643
Discovering JavaScript Object Notation IEEE Computer Society Computing Conversations Podcast None None 650971
Inventing PHP IEEE Computer Society Computing Conversations Podcast None None 494915
Monash Museum of Computing History IEEE Computer So

Walk Believer Walk The Black Crowes Warpaint Rock None None 280253
Choice Hotels International Undercover Boss Undercover Boss, Season 2 Reality TV 1 None 2614047
Microchip Jason Farnham YouTube Audio Library Dance & Electronic None None 114336
Anant Agarwal IEEE Computer Society Computing Conversations Podcast None None 494000
For Those About To Rock (We Salute You) AC/DC Who Made Who Rock
Hells Bells AC/DC Who Made Who Rock
Shake Your Foundations AC/DC Who Made Who Rock


In [2]:
import sqlite3
import json 


connection = sqlite3.connect('rosterdb.sqlite')
cursor = connection.cursor()

cursor.executescript('''
DROP TABLE IF EXISTS User ;
DROP TABLE IF EXISTS Course ;
DROP TABLE IF EXISTS Member ;

CREATE TABLE User (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name TEXT UNIQUE 

);

CREATE TABLE Course (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    title TEXT UNIQUE 
);

CREATE TABLE Member (
    user_id INTEGER ,
    course_id INTEGER ,
    role INTEGER ,
    PRIMARY KEY (user_id, course_id)


);

''')


fname = input('Enter : -')
if len(fname) < 1 :
    fname = 'data/roster_data_sample.json'
    
file = open(fname).read()
json_file = json.loads(file)


for entry in json_file :
    name = entry[0]
    title = entry[1]
    role = entry[2]
    print(name, title, role )
    
    cursor.execute('INSERT OR IGNORE INTO User (name) VALUES (?)', (name, ))
    cursor.execute('SELECT id FROM User WHERE name = ? ',(name,))
    user_id = cursor.fetchone()[0]
    
    cursor.execute('INSERT OR IGNORE INTO Course (title) VALUES (?)', (title, ))
    cursor.execute('SELECT id FROM Course WHERE title = ? ',(title,))
    course_id = cursor.fetchone()[0]
    
    cursor.execute('''INSERT OR REPLACE INTO Member 
    (user_id, course_id ,role) VALUES (?,?,?)''',(user_id,course_id, role))
    
    
    connection.commit()
    
sql_stat = '''SELECT 'XYZZY' || hex(User.name || Course.title || Member.role ) AS X FROM 
    User JOIN Member JOIN Course 
    ON User.id = Member.user_id AND Member.course_id = Course.id
    ORDER BY X LIMIT 1;'''

for col in cursor.execute(sql_stat):
    print(col)
    

Enter : -roster_data.json
Barrie si110 1
Chelsi si110 0
Safiyyah si110 0
Arfa si110 0
Ismail si110 0
Joelle si110 0
Clara si110 0
Lliam si110 0
Carris si110 0
Amir si110 0
Harlee si110 0
Aaima si110 0
Sinali si110 0
Corrin si110 0
Guinevere si110 0
Stanislaw si110 0
Jordyn si110 0
Dermot si110 0
Aran si110 0
Ruairi si110 0
Evan si110 0
Jameil si110 0
Lyena si110 0
Alber si110 0
Kemi si110 0
Corin si110 0
Dorothy si110 0
Aria si110 0
Mylo si110 0
Feden si110 0
Aayan si110 0
Bentley si110 0
Delilah si110 0
Inemesit si110 0
Breeanna si110 0
Amalie si110 0
Joy si106 1
Codie si106 0
Hollyanne si106 0
Jean si106 0
Karleigh si106 0
Rameen si106 0
Jean si106 0
Artemis si106 0
Khansa si106 0
Kealcy si106 0
Jackie si106 0
Ossian si106 0
Daryn si106 0
Tye si106 0
Plamedie si106 0
Luna si106 0
Latif si106 0
Mazin si106 0
Sebastien si106 0
Niomi si106 0
Elysse si206 1
Kristen si206 0
Bodhan si206 0
Kenzi si206 0
Alanas si206 0
Hollye si206 0
Ryszard si206 0
Ameerah si206 0
Oluwatobiloba si206 0
Mar

In [ ]:
import urllib.request, urllib.parse, urllib.error 
import http 
import ssl 
import json 
import sqlite3 
import time
import sys 



api_key = False 

if api_key is False:
    api_key = 42
    serviceurl = "http://py4e-data.dr-chuck.net/json?"
else :
    serviceurl = "https://maps.googleapis.com/maps/api/geocode/json?"

# make a  database     
conn = sqlite3.connect('geodata.sqlite')
cur = conn.cursor()

# create Locations table    
# will have some address and cache geograpical data cache the json
cur.execute('''

CREATE TABLE IF NOT EXISTS Locations (address TEXT, geodata TEXT)

''')

# ignore ssl certificate errors 
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

# open the file 
fh = open('where.data')
count = 0 


#loop thru the file 
for line in fh :
    if count > 200 :
        print('Retrieved 200 locations, restart to retrieve more')
        break
 # pull out the address        
    address = line.strip()
    print('')
# memoryview objects allow Python code to access the internal data of an object that supports the buffer protocol without copying.
# if its already there in the DB we dont wanna do it (restartable code)
    cur.exexecute('SELECT geodata FROM Locations WHERE address=?',(mmemoryview(address.encode()), ))
    
    try :
        data = cur.fetchone()[0]
        print("Found in database ",address)
        continue  
    except :
        pass       # otherwise keep going
    parms = dict()
    
# in url https://maps.googleapis.com/maps/api/geocode/json?query=American University in Cairo
    parms['query']= address   
    
    if api_key is not False :
        parms['key'] = api_key 
    
    url = serviceurl + urllib.parse.urlencode(params)
    print('Retrieving', url)
    
    uh = urllib.request.urlopen(url, context= ctx)
    data = uh.read().decode()
    print('Retrieved', len(data), 'characters', data[:20].replace('\n', ' '))
    count = count + 1
    
    try:
        js = json.loads(data)
        
    except :
        print(data)
        continue 
        
    if 'status' not in js or (js['status'] != 'OK' and js['status'] != 'ZERO_RESULTS' ):
    print('==== Failure To Retrieve ====')
        print(data)
        break
        
    cur.execute('''INSERT INTO Locations (address, geodata)
            VALUES ( ?, ? )''', (memoryview(address.encode()), memoryview(data.encode()) ) )
    conn.commit()
    
    # pause execution for 5 seconds 
    if count % 10 == 0 :
        print('Pausing for a bit ....')
        time.sleep(5)
        
print("Run geodump.py to read the data from the database so you can vizualize it on a map.")
        

In [ ]:
# then lets run this 

import sqlite3
import json
import codecs

conn = sqlite3.connect('geodata.sqlite')
cur = conn.cursor()

cur.execute('SELECT * FROM Locations')
fhand = codecs.open('where.js', 'w', "utf-8")
fhand.write("myData = [\n")
count = 0
for row in cur :
    data = str(row[1].decode())
    try: js = json.loads(str(data))
    except: continue

    if not('status' in js and js['status'] == 'OK') : continue

    lat = js["results"][0]["geometry"]["location"]["lat"]
    lng = js["results"][0]["geometry"]["location"]["lng"]
    if lat == 0 or lng == 0 : continue
    where = js['results'][0]['formatted_address']
    where = where.replace("'", "")
    try :
        print(where, lat, lng)

        count = count + 1
        if count > 1 : fhand.write(",\n")
        output = "["+str(lat)+","+str(lng)+", '"+where+"']"
        fhand.write(output)
    except:
        continue

fhand.write("\n];\n")
cur.close()
fhand.close()
print(count, "records written to where.js")
print("Open where.html to view the data in a browser")

